In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'
# TIP: look inside the data folders for more fine-graned conditions

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['ment','rnd']}
}

> For a detailed description of the tasks have a look pages 45-54 of the [HCP reference manual](https://www.humanconnectome.org/storage/app/media/documentation/s1200/HCP_S1200_Release_Reference_Manual.pdf).

In [ ]:
# @title Download data file
import os, requests

fname = "hcp_task.tgz"
url = "https://osf.io/2y3fw/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp_task"

# importing the "tarfile" module
import tarfile

# open file
with tarfile.open(fname) as tfile:
  # extracting file
  tfile.extractall('.')

subjects = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')

In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

In [ ]:
all_ROI_names = region_info['name'][1:]
err_txt = "I trained without the 1st column"
assert len(all_ROI_names) == 359, err_txt
print(all_ROI_names)
#print(region_info['network'])
#print(region_info['hemi'])

In [ ]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)
    # WHY?

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


# print(EXPERIMENTS)
# start computes start time in terms of frames, divides onset times by repitition time TR, round to integer
# duration computes length of trial in terms of frames by dividing duration time by TR, round up to integer
# frames = for each trial, generate range of frames corresponding to trial duration and start time..
def load_evs(subject, experiment, run):
  """Load EVs (explanatory variables) data for one task experiment.

  Args:
    subject (str): subject ID to load
    experiment (str) : Name of experiment
    run (int): 0 or 1

  Returns
    evs (list of lists): A list of frames associated with each condition

  """
  frames_list = []
  task_key = f'tfMRI_{experiment}_{RUNS[run]}'
  for cond in EXPERIMENTS[experiment]['cond']:
    ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)
    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)
    # Take the range of frames that correspond to this specific trial
    print(start)
    print(start.shape)
    print(duration)
    print(duration.shape)
    print(list(zip(start,duration)))
    frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
    print(frames)
    frames_list.append(frames)

  return frames_list

## Let's try with emotion...

In [ ]:
my_exp = 'EMOTION'
my_subj = subjects[1]
my_run = 1

data = load_single_timeseries(subject=my_subj,
                              experiment=my_exp,
                              run=my_run,
                              remove_mean=True)
print(data.shape)

(360, 176)


In [ ]:
evs = load_evs(subject=my_subj, experiment=my_exp, run=my_run)

In [ ]:
# A function to average all frames from any given condition

def average_frames3(data, evs, experiment, cond):
    # Find the index of the given condition within the experiment
    idx = EXPERIMENTS[experiment]['cond'].index(cond)

    # List to store the mean data for each set of event frames
    mean_data_list = []

    # Iterate over each set of event frames for the given condition
    for i in range(len(evs[idx])):
        # Debugging print statements
        print(f"\nProcessing event frame set {i + 1}/{len(evs[idx])} for condition '{cond}'")
        print(f"Event indices: {evs[idx][i]}")
        print(f"Data shape before extraction: {data.shape}")

        try:
            # Extract the data corresponding to the current set of event frames
            current_data = data[:, evs[idx][i]]
            print(f"Extracted data shape: {current_data.shape}")
        except IndexError as e:
            print(f"IndexError: {e}")
            continue

        # Compute the mean of the extracted data along the time axis
        mean_current_data = np.mean(current_data, axis=1, keepdims=True)
        print(f"Mean current data shape: {mean_current_data.shape}")

        # Append the mean data to the list
        mean_data_list.append(mean_current_data)

    if not mean_data_list:
        print("Error: No valid event frames found")
        return None

    stacked_means = np.stack(mean_data_list, axis=-1)
    print(f"Stacked means shape: {stacked_means.shape}")

    #return overall_mean
    return stacked_means

fr_activity = average_frames3(data, evs, my_exp, 'fear')
nt_activity = average_frames3(data, evs, my_exp, 'neut')
#contrast = fr_activity - nt_activity  # difference between 'fear' and 'neutral' conditions
if fr_activity is not None and nt_activity is not None:
    final_concatenated_means = np.concatenate((fr_activity, nt_activity), axis=-1)
    print(f"Final concatenated means shape: {final_concatenated_means.shape}")


In [ ]:
all_trials = []
all_labels = []

for i in range(len(subjects)):
    for r in [0, 1]:
        data = load_single_timeseries(subject=subjects[i],
                                      experiment=my_exp,
                                      run=r,
                                      remove_mean=True)

        # Get the trials for both conditions
        neut_trials = average_frames3(data, evs, my_exp, 'neut')
        fear_trials = average_frames3(data, evs, my_exp, 'fear')

        # Append each trial individually, flattened
        if neut_trials is not None:
            for trial in neut_trials.transpose(2, 0, 1):  # Iterate over trials and reshape
                all_trials.append(trial.flatten())
                all_labels.append(0)
        if fear_trials is not None:
            for trial in fear_trials.transpose(2, 0, 1):  # Iterate over trials and reshape
                all_trials.append(trial.flatten())
                all_labels.append(1)

# Convert to numpy array
all_trials_array = np.array(all_trials)

# Write trials and labels to separate csv files
np.savetxt('all_trials.csv', all_trials_array, delimiter=',')
np.savetxt('all_labels.csv', all_labels, delimiter=',')

#Begin Deep Neural Network

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix
from sklearn.metrics import *

#-- Pytorch specific libraries import -----#
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Make it more complicated!

In [ ]:
# write a foor loop o iterate over 360 input features (brain regions)
df_trials = pd.read_csv("./all_trials.csv")
df_labels = pd.read_csv("./all_labels.csv")

In [ ]:
#Train & Test Set
X= df_trials.iloc[: , :-1]
y= df_labels.iloc[: , -1] #target at the end column

train_x,test_x,train_y,test_y = train_test_split(X,y,random_state=42,test_size=0.2)
print("\n--Training data samples--")
print(train_x.shape)
print(test_x.shape)
print("\n--Testing data samples--")
print(train_y.shape)
print(test_y.shape)

In [ ]:
###First use a MinMaxscaler to scale all the features of Train & Test dataframes

scaler = preprocessing.MinMaxScaler() #normalizes the features
x_train = scaler.fit_transform(train_x.values)
x_test =  scaler.fit_transform(test_x.values)

#print("Scaled values of Train set \n")
#print(x_train)
#print("\nScaled values of Test set \n")
#print(x_test)


###Then convert the Train and Test sets into Tensors

x_tensor =  torch.from_numpy(x_train).float() #converts numpy array to pytorch tensor
y_tensor =  torch.from_numpy(train_y.values.ravel()).float() #flattens the array into a one-dimensional array
xtest_tensor =  torch.from_numpy(x_test).float()
ytest_tensor =  torch.from_numpy(test_y.values.ravel()).float() #flattens the array into a one-dimensional array

#print("\nTrain set Tensors \n")
#print(x_tensor)
#print(y_tensor)
#print("\nTest set Tensors \n")
#print(xtest_tensor)
#print(ytest_tensor)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

class ChurnModel(nn.Module):
    def __init__(self, n_input_dim):
        super(ChurnModel, self).__init__()
        self.n_hidden1 = 720
        self.n_hidden2 = 720
        self.n_output = 1
        self.layer_1 = nn.Linear(n_input_dim, self.n_hidden1)
        self.layer_2 = nn.Linear(self.n_hidden1, self.n_hidden2)
        self.layer_out = nn.Linear(self.n_hidden2, self.n_output)


        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(self.n_hidden1)
        self.batchnorm2 = nn.BatchNorm1d(self.n_hidden2)


    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))

        return x


#model = ChurnModel(n_input_dim)
#print(model)

In [ ]:
# This loss function is typically used for binary classification tasks where the model outputs probabilities (e.g., values between 0 and 1)
loss_func = nn.BCELoss()

In [ ]:
y_true_test = test_y.values.ravel() #This method flattens the array into a one-dimensional format.
conf_matrix = confusion_matrix(y_true_test ,ytest_pred)
print("Confusion Matrix of the Test Set")
print("-----------")
print(conf_matrix)
#Precision is defined as the ratio of true positive predictions to the total predicted positives
print("Precision of the MLP :\t"+str(precision_score(y_true_test,ytest_pred)))
print("Recall of the MLP    :\t"+str(recall_score(y_true_test,ytest_pred)))
print("F1 Score of the Model :\t"+str(f1_score(y_true_test,ytest_pred)))

#Random Sampling

In [ ]:
#Define a batch size ,
bs = 32
x_tensor =  torch.from_numpy(x_train).float() #converts numpy array to pytorch tensor
y_tensor =  torch.from_numpy(train_y.values.ravel()).float() #flattens the array into a one-dimensional array
xtest_tensor =  torch.from_numpy(x_test).float()
ytest_tensor =  torch.from_numpy(test_y.values.ravel()).float() #flattens the array into a one-dimensional array
y_tensor = y_tensor.unsqueeze(1)

#For the validation/test dataset
ytest_tensor = ytest_tensor.unsqueeze(1)
test_ds = TensorDataset(xtest_tensor, ytest_tensor)
test_loader = DataLoader(test_ds, batch_size=32)#model will process 32 samples at a time
accuracies = []
#num_ROIs = x_tensor.shape[1]
new_x_tensor = x_tensor#[:, indices][:, second_indices][:, third_indices][:, fourth_indices][:, fifth_indices][:, sixth_indices][:, seventh_indices][:, eigth_indices]
print(new_x_tensor)
num_ROIs = new_x_tensor.shape[1]
new_xtest_tensor = xtest_tensor#[:, indices][:, second_indices][:, third_indices][:, fourth_indices][:, fifth_indices][:, sixth_indices][:, seventh_indices][:, eigth_indices]
print(num_ROIs)
epochs = 50


random_best_accs = []

#200 randome samples
for i in range(200):
  # choose random indices from the 359 original brain regions with the length of 6
  random_indices = np.random.choice(num_ROIs, size=6, replace=False)
  # first seven brain regions (i.e. ~random)
  # x_subset = new_x_tensor[:, :7]
  # last seven brain regions (i.e. ~random)
  # x_subset = new_x_tensor[:, -7:]
  x_subset = new_x_tensor[:, random_indices]
  # create a tensor dataset with the new subset of x
  train_ds = TensorDataset(x_subset, y_tensor)
  train_dl = DataLoader(train_ds, batch_size=bs)
  test_ds = TensorDataset(new_xtest_tensor[:, random_indices], ytest_tensor)
  test_loader = DataLoader(test_ds, batch_size=32)
  n_input_dim = x_subset.shape[1]
  model = ChurnModel(n_input_dim)
  # 0.001 lr for everything before fifth_indices
  optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

  train_loss = []
  val_acc = []
  for epoch in range(epochs):
      model.train()
      # Within each epoch run the subsets of data = batch sizes
      epoch_loss = 0
      for xb, yb in train_dl:
          y_pred = model(xb)            # Forward Propagation
          loss = loss_func(y_pred, yb)  # Loss Computation
          epoch_loss += loss.item()
          optimizer.zero_grad()         # Clearing all previous gradients, setting to zero
          loss.backward()               # Back Propagation
          optimizer.step()              # Updating the parameters
      #print(f"Loss @ Epoch #{epoch}: {epoch_loss:.4f}")
      train_loss.append(epoch_loss)
      # get validation accuracy for each epoch
      v_acc = evaluate_model(model, test_loader)
      val_acc.append(v_acc)

  # we are getting the maximum test accuracy from all 50 training epochs for each model
  accuracy = max(val_acc)
  random_best_accs.append(accuracy)
  save_file = r'/content/drive/My Drive/random_6_mlp_acc.npy'
  np.save(save_file, random_best_accs)

  #print(f'{len(accuracies)} models trained! c:')
  print("ACCURACY", accuracy)
  plt.figure()
  plt.plot(train_loss, label='Training Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.title(f'Model #{i}')

  plt.figure()
  plt.plot(val_acc, label='Validation Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.title(f'Model #{i}')

  if i % 5 == 0 and i != 0:
    plt.figure()
    plt.hist(random_best_accs)

  plt.show()

In [ ]:
plt.figure()
plt.hist(random_best_accs)
plt.title('Random Sampling for 6 Brain Regions')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.show()

#Automate the process of getting new indices from the subset of the data

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for xb, yb in test_loader:
            y_pred = model(xb)  # Get model predictions
            y_pred_tag = torch.round(y_pred)  # Round predictions to get binary outputs
            correct += (y_pred_tag.eq(yb).sum().item())  # Count correct predictions
            total += yb.size(0)  # Count total samples

    accuracy = correct / total  # Calculate accuracy
    return accuracy

#Final Sampling - Automated

In [ ]:
from google.colab import drive
import torch

def get_indices_from_file (file_path, cutoff):
  data = np.load(file_path)
  indices_bool = (data >= cutoff)
  indices_bool = np.invert(indices_bool)
  indices = np.where(indices_bool)[0]
  return indices


drive.mount('/content/drive')

#target_num_ROIs = int(2/3*num_ROIs)
num_ROIs = x_tensor.shape[1]

def find_optimal_cutoffs(file_path):
  accuracy_data = np.load(file_path)
  cutoff = 1
  step = 0.005
  num_ROIs = len(accuracy_data)
  target_num_ROIs = int(2/3*num_ROIs)

  while num_ROIs > target_num_ROIs:
    cutoff -= step
    num_ROIs = len(get_indices_from_file(file_path, cutoff))

  #check if previous cutoff was closer to the target
  prev_num_ROIs = len(get_indices_from_file(file_path, cutoff+step))
  if abs(prev_num_ROIs - target_num_ROIs) < abs(num_ROIs - target_num_ROIs):
    cutoff += step

  return cutoff

bs = 32
x_tensor =  torch.from_numpy(x_train).float() #converts numpy array to pytorch tensor
y_tensor =  torch.from_numpy(train_y.values.ravel()).float() #flattens the array into a one-dimensional array
xtest_tensor =  torch.from_numpy(x_test).float()
ytest_tensor =  torch.from_numpy(test_y.values.ravel()).float() #flattens the array into a one-dimensional array
y_tensor = y_tensor.unsqueeze(1)

#For the validation/test dataset
ytest_tensor = ytest_tensor.unsqueeze(1)
test_ds = TensorDataset(xtest_tensor, ytest_tensor)
test_loader = DataLoader(test_ds, batch_size=32)#model will process 32 samples at a time
accuracies = []

#TODO remove-this is just for debugging
#x_tensor = x_tensor[:, :6]; num_rounds = 2

num_rounds = 9 #eventually i wanna do it for 8 rounds just like the original code
for k in range(num_rounds):
  # explicity make sure we have a copy of the tensor
  new_x_tensor = x_tensor.detach().clone()#.to('cuda')
  new_xtest_tensor = xtest_tensor.detach().clone()#.to('cuda')
  ROI_names = np.array(all_ROI_names.copy())
  # we want to do this unless it's the first time:
  # load the ith accuracy file, get its indices via cutoff function, then apply
  for l in range(k): # this for loop is for trim trim trim!!!
    file_path = f'/content/drive/My Drive/{l+1}_mlp_acc.npy'
    cutoff = find_optimal_cutoffs(file_path)
    indices = get_indices_from_file(file_path, cutoff)
    new_x_tensor = new_x_tensor[:, indices]
    new_xtest_tensor = new_xtest_tensor[:, indices]
    ROI_names = ROI_names[indices]
    print(ROI_names)

  num_ROIs = new_x_tensor.shape[1]
  accuracies = []
  for i in range(num_ROIs):
    ROI_indices = [j for j in range(num_ROIs) if j!=i]
    #select a subset of brain regions with highest accuracies
    x_subset = new_x_tensor[:, ROI_indices]
    #x_subset = x_tensor[:,ROI_indices]
    #create a tensor dataset with the new subset of x
    #train_ds = TensorDataset(x_subset.to('cuda'), y_tensor.to('cuda'))
    train_ds = TensorDataset(x_subset, y_tensor)
    train_dl = DataLoader(train_ds, batch_size=bs)
    #test_ds = TensorDataset(new_xtest_tensor[:,ROI_indices], ytest_tensor.to('cuda'))
    test_ds = TensorDataset(new_xtest_tensor[:,ROI_indices], ytest_tensor)
    test_loader = DataLoader(test_ds, batch_size=32)
    n_input_dim = x_subset.shape[1]
    model = ChurnModel(n_input_dim)
    #model = model.to('cuda')
    # 0.001 for everything before fifth_indices
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

    train_loss = []
    val_acc = []
    epochs = 50
    for epoch in range(epochs):
        model.train()
        #Within each epoch run the subsets of data = batch sizes.
        epoch_loss = 0
        for xb, yb in train_dl:
            #print(xb.shape)
            y_pred = model(xb)            # Forward Propagation
            #print(y_pred)
            #print(yb)
            loss = loss_func(y_pred, yb)  # Loss Computation
            epoch_loss += loss.item()
            optimizer.zero_grad()         # Clearing all previous gradients, setting to zero
            loss.backward()               # Back Propagation
            optimizer.step()              # Updating the parameters
        #print(f"Loss @ Epoch #{epoch}: {epoch_loss:.4f}")
        train_loss.append(epoch_loss)
        # get validation accuracy
        v_acc = evaluate_model(model, test_loader)
        val_acc.append(v_acc)

    # we are getting the maximum test accuracy from all 50 training epochs for each model
    accuracy = max(val_acc)
    accuracies.append(accuracy)
    save_file = f'/content/drive/My Drive/{k+1}_mlp_acc.npy'
    np.save(save_file, accuracies)
    print(f'{len(accuracies)} models trained! c:')
    print("ACCURACY", accuracy)
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Model #{i}')

    plt.figure()
    plt.plot(val_acc, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title(f'Model #{i}')
    plt.show()

In [ ]:
for i in range(num_rounds): # this for loop is for trim trim trim!!!
    file_path = f'/content/drive/My Drive/{i+1}_mlp_acc.npy'
    accuracy_data = np.load(file_path)
    plt.figure()
    plt.hist(accuracy_data, label="Validation Accuracy")
    plt.legend()
    plt.title(f'Trim Round #{i+1} Results')
plt.show()

In [ ]:
# automate learning rate tuning

# train the n 'test' (e.g. 3) networks/models

# look at the validation accuracies over epochs / training

# make sure each network has at least k (e.g. 3) new best validation accuracies
# ('peaks') during the first j (e.g. 10) epochs

# if not, decrease learning rate by 2x (i.e. multiple by 0.5)
# and re-run the n 'test' networks.

# once these criteria are met, we can t rain all the networks, not just the n 'test' ones.